In [24]:
import os, sys
import numpy as np
import pandas as pd
from neo4j import GraphDatabase
host = 'bolt://localhost:7687'
user = 'neo4j'
password = 'LimeStardom6J'
driver = GraphDatabase.driver(host,auth=(user, password))

def run_query(query):
    with driver.session() as session:
        result = session.run(query)
        return pd.DataFrame([r.values() for r in result], columns=result.keys())

In [25]:
run_query("""
LOAD CSV WITH HEADERS FROM "file:///crypto_prices.csv" as row
CALL {
    WITH row
    MERGE (s:Crypto{name:row.Name})
    CREATE (s)-[:TRADING_DAY]->(:CryptoTradingDay{date: date(row.Date), close:toFloat(row.Close), volume: toFloat(row.Volume)})
} IN TRANSACTIONS
""")

""


In [26]:
run_query("""
MATCH (s:Crypto)-[:TRADING_DAY]->(day)
WITH s, day
ORDER BY day.date ASC
WITH s, collect(day) as nodes, collect(day.close) as closes
SET s.close_array = closes
WITH nodes
CALL apoc.nodes.link(nodes, 'NEXT_DAY')
RETURN distinct 'done' AS result
""")

,result
0,done


In [27]:
run_query("""
CALL gds.graph.project('crypto', 'Crypto', '*', {nodeProperties:['close_array']})
""")

,nodeProjection,relationshipProjection,graphName,nodeCount,relationshipCount,projectMillis
0,"{'Crypto': {'label': 'Crypto', 'properties': {...","{'__ALL__': {'aggregation': 'DEFAULT', 'orient...",crypto,52,0,825


In [28]:
run_query("""
CALL gds.knn.mutate('crypto', 
  {nodeProperties:'close_array', topK:3, similarityCutoff: 0.2, mutateProperty:'score', mutateRelationshipType:'SIMILAR'})
""")

,ranIterations,nodePairsConsidered,didConverge,preProcessingMillis,computeMillis,mutateMillis,postProcessingMillis,nodesCompared,relationshipsWritten,similarityDistribution,configuration
0,5,4375,True,0,82,53,0,52,156,"{'min': 0.9977798461914062, 'p5': 0.9983367919...","{'mutateProperty': 'score', 'jobId': '3f919e75..."


In [29]:
run_query("""
CALL gds.louvain.write('crypto', {
  relationshipTypes:['SIMILAR'],
  writeProperty:'louvain'
})
""")

,writeMillis,nodePropertiesWritten,modularity,modularities,ranLevels,communityCount,communityDistribution,postProcessingMillis,preProcessingMillis,computeMillis,configuration
0,26,52,0.54931,[0.54930966469428],1,8,"{'min': 5, 'p5': 5, 'max': 9, 'p999': 9, 'p99'...",9,0,107,"{'writeProperty': 'louvain', 'jobId': '335a47d..."


In [30]:
run_query("""
MATCH (s:Crypto)-[:TRADING_DAY]->(day)
CALL apoc.create.addLabels(day, [s.name]) YIELD node
RETURN distinct 'done'
""")


,'done'
0,done


In [31]:
run_query("""
MATCH (s:Crypto)-[:TRADING_DAY]->(day)
WHERE NOT exists { ()-[:NEXT_DAY]->(day) }
MATCH p=(day)-[:NEXT_DAY*0..]->(next_day)
SET next_day.index = length(p)
""")

""


In [32]:
run_query("""
MATCH (s:Crypto)
CALL apoc.math.regr(s.name, 'close', 'index') YIELD slope
SET s.slope = slope;
""")

""


In [33]:
run_query("""
MATCH (s:Crypto)
WITH s.louvain AS community, s.slope AS slope, s.name AS ticker
ORDER BY slope DESC
RETURN community, collect(ticker)[..5] as potential_investments
""")

,community,potential_investments
0,22,"[BTC, LTC, INJ, ATOM, STX]"
1,2,"[ETH, SOL, ETC, LINK, OP]"
2,19,"[BNB, ORDI, APT, XAI, RUNE]"
3,24,"[TRB, WLD, FIL, NFP, FTM]"
4,18,"[SSV, TIA, ENS, LDO, NTRN]"
5,17,"[AVAX, DOT, JTO, MATIC, ADA]"
6,30,"[ACE, DYDX, GMT, CHZ, MEME]"
7,49,"[ICP, ARB, AI, SEI, BLUR]"
